In [1]:
from pyspark.sql import SparkSession, functions as F

spark = (
    SparkSession.builder.appName("Regression")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.executor.memory", "2g")
    .config("spark.driver.memory", "4g")
    .getOrCreate()
)


22/09/18 14:26:12 WARN Utils: Your hostname, DESKTOP-EMCMOU9 resolves to a loopback address: 127.0.1.1; using 172.20.180.85 instead (on interface eth0)
22/09/18 14:26:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/18 14:26:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# read in datasets
data = spark.read.parquet('../data/curated/cleaned_data.parquet')
merchant_fraud = spark.read.parquet('../data/curated/merchant_fraud.parquet')
consumer_fraud = spark.read.parquet('../data/curated/consumer_fraud.parquet')

In [3]:
data.limit(15)

22/09/17 14:33:08 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


SA2_CODE_2021,postcode,consumer_id,user_id,merchant_abn,order_datetime,dollar_value,order_id,is_fraud,fraud_probability,name,tags,take rate,revenue level,consumer_name,address,gender,SA2_NAME_2021,Median_tot_prsnl_inc_weekly,Median_rent_weekly,Median_mortgage_repay_monthly,Median_age_persons,Median_tot_hhd_inc_weekly,Average_household_size,Year_12_Highest_Level_of_School,Did_Not_Attend_School,TOT_P_P,state,date,year,take_rate*dollar_value
405041124,5554,101,9180,75944642726,2021-08-20,41.621353566140854,4e80e6a1-bdc5-4b5...,false,0.01,Et PC,[tent and awning ...,6.03,a,Jasmine Sanchez,598 Johnson Motor...,Female,Kadina,599.0,250.0,1127.0,45.0,1155.0,2.299999952316284,1522.0,13.0,5414.0,SA,08-20,2021,250.97676200382935
405041124,5554,101,9180,31584328627,2021-08-20,41.012145786222305,42b8acb6-1263-453...,false,0.01,null,null,null,null,Jasmine Sanchez,598 Johnson Motor...,Female,Kadina,599.0,250.0,1127.0,45.0,1155.0,2.299999952316284,1522.0,13.0,5414.0,SA,08-20,2021,null
405041124,5554,101,9180,45629217853,2021-08-20,6.7656818985097535,f107384a-29eb-468...,false,0.01,Lacus Consulting,"[gift, card, no...",6.98,a,Jasmine Sanchez,598 Johnson Motor...,Female,Kadina,599.0,250.0,1127.0,45.0,1155.0,2.299999952316284,1522.0,13.0,5414.0,SA,08-20,2021,47.22445965159808
405041124,5554,101,9180,71989214810,2021-08-21,174.08192180474757,e1dbffda-f291-41d...,false,0.01,Semper Et Lacinia...,[computer program...,6.95,a,Jasmine Sanchez,598 Johnson Motor...,Female,Kadina,599.0,250.0,1127.0,45.0,1155.0,2.299999952316284,1522.0,13.0,5414.0,SA,08-21,2021,1209.8693565429955
405041124,5554,101,9180,68216911708,2021-08-21,12.442459813951164,24b53c84-dd33-48c...,false,0.01,Placerat Eget Ven...,"[computers, comp...",3.05,c,Jasmine Sanchez,598 Johnson Motor...,Female,Kadina,599.0,250.0,1127.0,45.0,1155.0,2.299999952316284,1522.0,13.0,5414.0,SA,08-21,2021,37.94950243255105
405041124,5554,101,9180,64203420245,2021-08-22,47.48856801925245,cb50ad85-e8f0-42c...,false,0.01,Pede Nonummy Corp.,[tent and awning ...,2.86,c,Jasmine Sanchez,598 Johnson Motor...,Female,Kadina,599.0,250.0,1127.0,45.0,1155.0,2.299999952316284,1522.0,13.0,5414.0,SA,08-22,2021,135.817304535062
405041124,5554,101,9180,38212167834,2021-08-22,548.9214138172234,53fd2495-2cef-491...,false,0.01,Magna Suspendisse...,[lawn and garden ...,3.83,b,Jasmine Sanchez,598 Johnson Motor...,Female,Kadina,599.0,250.0,1127.0,45.0,1155.0,2.299999952316284,1522.0,13.0,5414.0,SA,08-22,2021,2102.3690149199656
405041124,5554,101,9180,17523010120,2021-08-15,2.3742914221884552,f0ec9a69-dc62-464...,false,0.01,Urna Convallis Fo...,[tent and awning ...,0.79,d,Jasmine Sanchez,598 Johnson Motor...,Female,Kadina,599.0,250.0,1127.0,45.0,1155.0,2.299999952316284,1522.0,13.0,5414.0,SA,08-15,2021,1.8756902235288797
405041124,5554,101,9180,32844138686,2021-07-15,369.5165979783281,3feade93-80b4-49b...,false,0.01,At Augue Id Assoc...,[computer program...,1.89,c,Jasmine Sanchez,598 Johnson Motor...,Female,Kadina,599.0,250.0,1127.0,45.0,1155.0,2.299999952316284,1522.0,13.0,5414.0,SA,07-15,2021,698.3863701790401
405041124,5554,101,9180,51279178333,2021-07-15,34.72663298338204,c4ef84a1-4d13-4c2...,false,0.01,Neque LLC,[music shops - mu...,1.83,c,Jasmine Sanchez,598 Johnson Motor...,Female,Kadina,599.0,250.0,1127.0,45.0,1155.0,2.299999952316284,1522.0,13.0,5414.0,SA,07-15,2021,63.549738359589135


In [3]:
merchant_fraud.limit(10)

merchant_abn,order_datetime,transaction_count,avg_transaction_amt,is_fraud,fraud_probability
38700038932,2021-08-20,14,1524.9292206627556,false,0.01
89502033586,2021-08-20,40,65.44754598876243,false,0.01
70610974780,2021-08-20,27,43.48907978704201,false,0.01
29550468444,2021-08-20,4,157.0,false,0.01
92075595936,2021-08-20,12,95.50018824235849,false,0.01
62422038748,2021-08-20,1,2442.1698743679544,false,0.01
32894483068,2021-08-20,1,228.6561829721828,false,0.01
55958301764,2021-08-20,6,281.354343839723,false,0.01
12543580354,2021-08-20,12,262.53675367314503,false,0.01
26008308191,2021-08-20,6,241.04606090655918,false,0.01


In [9]:
merchant_fraud.where(F.col('fraud_probability') != 0.01)

merchant_abn,order_datetime,transaction_count,avg_transaction_amt,is_fraud,fraud_probability
83220249221,2021-11-27,6,157.8399073660867,true,63.51245947057463
18158387243,2021-11-29,77,558.986275523425,true,28.956947892226466
22606727520,2021-11-27,5,394.6312098891729,true,62.992786990153064
31334588839,2021-12-11,6,9032.604983300089,true,31.415759105830013
31334588839,2021-12-10,7,6221.920428612912,true,26.940495859428125
19492220327,2021-12-18,4,16149.939551642537,true,33.819672154331755
14530561097,2021-09-15,1,46001.13901942742,true,80.80054474543395
97884414539,2021-10-19,1,44615.02252113347,true,89.79919971536573
15043504837,2021-08-29,1,47381.93709086238,true,59.77648897297805
30275989942,2022-01-28,1,1558.8759753180334,true,63.108592876362856


In [12]:
merchant_fraud.where(F.col('fraud_probability') != 0.01).count()

228

In [10]:
merchant_fraud.describe()

summary,merchant_abn,transaction_count,avg_transaction_amt,fraud_probability
count,2689030,2689030,2689030,2689030
mean,5.424660427391621E10,10.558086001271834,461.60592274364814,0.013426264605000475
stddev,2.586475476711026E10,31.94851692671459,1304.331852780586,0.40406670296903746
min,10023283211,1,7.323788293697512E-4,0.01
max,99990536339,1260,105193.88578925544,94.1347004808891


In [4]:
consumer_fraud.limit(10)

user_id,order_datetime,transaction_count,avg_transaction_amt,is_fraud,fraud_probability
18487,2021-08-20,2,73.0925761125627,false,0.01
18552,2021-08-20,2,174.42985218811984,false,0.01
215,2021-08-20,2,48.880342827706,false,0.01
18770,2021-08-20,1,273.2147324712209,false,0.01
18837,2021-08-20,1,23.800263170555642,false,0.01
18849,2021-08-20,3,56.899667578706065,false,0.01
435,2021-08-20,2,21.310166954483677,false,0.01
788,2021-08-20,2,38.272827709859286,false,0.01
19550,2021-08-20,2,17.85711450212551,false,0.01
1117,2021-08-20,3,57.4104700047829,false,0.01


In [11]:
consumer_fraud.count()

22/09/18 14:36:41 WARN NettyRpcEnv: Ignored failure: java.util.concurrent.TimeoutException: Cannot receive any reply from 172.20.180.85:34969 in 10000 milliseconds
22/09/18 14:36:42 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.rpc.RpcTimeoutException: Futures timed out after [10000 milliseconds]. This timeout is controlled by spark.executor.heartbeatInterval
	at org.apache.spark.rpc.RpcTimeout.org$apache$spark$rpc$RpcTimeout$$createRpcTimeoutException(RpcTimeout.scala:47)
	at org.apache.spark.rpc.RpcTimeout$$anonfun$addMessageIfTimeout$1.applyOrElse(RpcTimeout.scala:62)
	at org.apache.spark.rpc.RpcTimeout$$anonfun$addMessageIfTimeout$1.applyOrElse(RpcTimeout.scala:58)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:76)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:103)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.

ERROR:root:KeyboardInterrupt while sending command.==========>      (8 + 1) / 9]
Traceback (most recent call last):
  File "/home/seenyanl/.local/lib/python3.8/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/seenyanl/.local/lib/python3.8/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.8/socket.py", line 669, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
import pandas as pd
from sklearn.utils import resample

def resampling(majority, minority):
    """
    Performs undersampling on the majority class 
    and oversampling on the minority class from an imbalanced dataset. 
    Outputs resampled dataset as a pandas dataframe.
    """
    
    # define number of samples needed from each class
    N_MAJOR = 550
    N_MINOR = 450
    
    # repeated sampling from the minority class
    n_more_samples = N_MINOR - minority.count() # needs to be less than no. of samples in minority
    sampled_minority = minority.union(resample(minority, replace=False, n_samples=n_more_samples, random_state=0)).toPandas()
    
    # undersampling from the majority class
    sampled_majority = majority.sample(N_MAJOR, set_seed=0).toPandas()
    
    # join the sampled datasets
    resampled = pd.concat([sampled_majority, sampled_minority], axis=0)
    
    return resampled

In [ ]:
# split merchant fraud data by class
merchant_majority = merchant_fraud.where(F.col('fraud_probability') == 0.01)
merchant_minority = merchant_fraud.where(F.col('fraud_probability') != 0.01)

# obtain resampled dataset for modelling
merchant_resampled = resampling(merchant_majority, merchant_minority)

In [ ]:
# split dataset into predictors and target variables
X
y

In [ ]:
from sklearn.model_selection import StratifiedKFold

# initialise stratified sampling
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

# initialise model


for train_index, test_index in skf.split(X, y):
    # obtain train and test folds
    X_train_fold, X_test_fold = X[train_index], X[test_index]
    y_train_fold, y_test_fold = y[train_index], y[test_index]
    
    # fit the model and perform evaluation